In [ ]:
# Setup imports and configuration
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from typing import Optional, List

# Configuration - adjust if notebook is moved
LOG_DIR = Path('../logs')
NUM_LOGS_TO_PLOT = 1  # set >1 to overlay multiple recent logs

plt.style.use('seaborn-darkgrid')
print(f'Looking for logs in: {LOG_DIR.resolve()}')
print(f'Will plot {NUM_LOGS_TO_PLOT} most recent log(s)')

In [ ]:
# Helper functions
def get_latest_log(log_dir: Path) -> Optional[Path]:
    """Return the newest matching CSV path or None if none found."""
    if not log_dir.exists():
        return None
    files = list(log_dir.glob('balance_*.csv'))
    if not files:
        return None
    return max(files, key=lambda f: f.stat().st_mtime)

def get_latest_logs(log_dir: Path, n: int) -> List[Path]:
    if not log_dir.exists():
        return []
    files = sorted(list(log_dir.glob('balance_*.csv')), key=lambda f: f.stat().st_mtime, reverse=True)
    return files[:n]


In [ ]:
# Load single or multiple latest logs depending on NUM_LOGS_TO_PLOT
log_file: Optional[Path] = None
log_list = []

if NUM_LOGS_TO_PLOT == 1:
    log_file = get_latest_log(LOG_DIR)
    if log_file is None:
        print(f'No log files found in {LOG_DIR}')
    else:
        print(f'Loading: {log_file.name}')
        try:
            df = pd.read_csv(log_file)
            if df.empty:
                print(f'CSV {log_file} is empty')
            else:
                # Safe metadata extraction
                gains = {
                    'Kp': df['kp_balance'].iloc[0] if 'kp_balance' in df.columns and not df['kp_balance'].isna().all() else float('nan'),
                    'Ki': df['ki_balance'].iloc[0] if 'ki_balance' in df.columns and not df['ki_balance'].isna().all() else float('nan'),
                    'Kd': df['kd_balance'].iloc[0] if 'kd_balance' in df.columns and not df['kd_balance'].isna().all() else float('nan'),
                }
                reason = df['reason'].iloc[0] if 'reason' in df.columns else 'unknown'
                print(f'Gains: {gains} | End Reason: {reason}')
                # Normalize time
                if 'time_s' in df.columns:
                    df['t'] = df['time_s'] - df['time_s'].iloc[0]
                else:
                    df['t'] = range(len(df))
        except Exception as e:
            print(f'Failed to load CSV: {e}')

else:
    files = get_latest_logs(LOG_DIR, NUM_LOGS_TO_PLOT)
    if not files:
        print(f'No log files found in {LOG_DIR}')
    else:
        for f in files:
            try:
                dfi = pd.read_csv(f)
                if 'time_s' in dfi.columns and not dfi.empty:
                    dfi['t'] = dfi['time_s'] - dfi['time_s'].iloc[0]
                else:
                    dfi['t'] = range(len(dfi))
                log_list.append((f.name, dfi))
            except Exception as e:
                print(f'Skipping {f.name}: {e}')
        if log_list:
            df = log_list[0][1]
            gains = {
                'Kp': df['kp_balance'].iloc[0] if 'kp_balance' in df.columns and not df['kp_balance'].isna().all() else float('nan'),
                'Ki': df['ki_balance'].iloc[0] if 'ki_balance' in df.columns and not df['ki_balance'].isna().all() else float('nan'),
                'Kd': df['kd_balance'].iloc[0] if 'kd_balance' in df.columns and not df['kd_balance'].isna().all() else float('nan'),
            }
            reason = df['reason'].iloc[0] if 'reason' in df.columns else 'unknown'


In [ ]:
# Plot: System States
if 'df' in locals() and not df.empty:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    if 'pendulum_deg' in df.columns:
        ax1.plot(df['t'], df['pendulum_deg'], label='Pendulum Angle', color='tab:blue')
    ax1.axhline(0, color='k', linestyle='--', alpha=0.3)
    ax1.set_ylabel('Angle (deg)')
    ax1.set_title(f'Pendulum Stability')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    if 'base_deg' in df.columns:
        ax2.plot(df['t'], df['base_deg'], label='Base/Motor Angle', color='tab:orange')
    ax2.set_ylabel('Angle (deg)')
    ax2.set_xlabel('Time (s)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print('No data loaded; cannot plot system states')

In [ ]:
# Plot: Control Effort & Phase Space
if 'df' in locals() and not df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    if 'control_output' in df.columns:
        ax1.plot(df['t'], df['control_output'], color='tab:green')
        ax1.set_title('Control Effort (Stepper Hz)')
        ax1.set_xlabel('Time (s)')
        ax1.set_ylabel('Steps/Sec (Hz)')
        ax1.grid(True, alpha=0.3)
    if 'pendulum_deg' in df.columns and 'pendulum_vel' in df.columns:
        ax2.plot(df['pendulum_deg'], df['pendulum_vel'], color='purple', alpha=0.6)
        ax2.plot(df['pendulum_deg'].iloc[0], df['pendulum_vel'].iloc[0], 'go', label='Start')
        ax2.plot(df['pendulum_deg'].iloc[-1], df['pendulum_vel'].iloc[-1], 'rx', label='End')
        ax2.set_title('Phase Portrait (Pendulum)')
        ax2.set_xlabel('Angle (deg)')
        ax2.set_ylabel('Velocity (deg/s)')
        ax2.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print('No data loaded; cannot plot control effort')

In [ ]:
# Summary statistics
if 'df' in locals() and not df.empty:
    print('
=== SUMMARY ===')
    print(f'Duration: {df['t'].iloc[-1]:.2f} s')
    print(f'Samples: {len(df)}')
    if 'pendulum_deg' in df.columns:
        print(f'Pendulum RMS: {df['pendulum_deg'].std():.2f} deg')
    if 'control_output' in df.columns:
        print(f'Control RMS: {df['control_output'].std():.2f} Hz')
    if 'gains' in locals():
        print(f"Gains: Kp={gains['Kp']}, Ki={gains['Ki']}, Kd={gains['Kd']}")
    if 'reason' in locals():
        print(f'End Reason: {reason}')
else:
    print('No data loaded; cannot show summary')